# Getting sentiment from ChatGPT

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import openai
import redis
import asyncio
import backoff

In [2]:
# r = redis.Redis("localhost", 5211, decode_responses=True)
db = sqlite3.connect("scraping.db")

In [3]:
messages = [{
    "role": "system", 
    "content": "Read the article, list named entities discussed in it, choose a sentiment between positive, neutral, and negative for each listed entity separately. Provide your answer in a json format with keys 'name', 'sentiment', 'explanation'."
}]

with open("../key.txt", "r") as f:
    client = openai.AsyncOpenAI(
        api_key=f.read(),
    )

while True:
    cur = db.execute("select * from article where headline != 'None' and content != 'None' and content != '' and feedback is null and content like '%United States%' limit 64")
    db.commit()

    l = list(cur)
    if len(l) == 0:
        break

    async def get_sentiment(row):
        chat = await client.chat.completions.create( 
            model="gpt-3.5-turbo-1106", 
            response_format={ "type": "json_object" },
            messages=messages + [{"role": "user", "content": "Title: " + row[1] + "\nContent: " + row[3]}] 
        )
        reply = chat.choices[0].message.content

        return row[0], reply

    async def multiple_sentiments(rows):
        tasks = [get_sentiment(row) for row in rows]
        res = await asyncio.gather(*tasks, return_exceptions=True)
        return res

    res = await multiple_sentiments(l)
    print(res)
    for r in res:
        try:
            db.execute(f'update article set feedback = ? where id = ?', (r[1], r[0]))
        except TypeError as e:
            db.commit()
            cur = db.execute("select count(*) from article where feedback is not null")
            db.commit()
            print(f"Sleeping... got {list(cur)[0][0]} feedbacks!")
            time.sleep(61)
    db.commit()

[(1066079, '{\n  "entities": [\n    {\n      "name": "Karen Donfried",\n      "sentiment": "positive",\n      "explanation": "Karen Donfried expressed readiness of the United States in supporting the sides in the normalization of Armenian-Azerbaijani relations."\n    },\n    {\n      "name": "Armenia",\n      "sentiment": "neutral",\n      "explanation": "Armenia is mentioned in the context of normalization of relations with Azerbaijan, hence the sentiment is neutral."\n    },\n    {\n      "name": "Azerbaijan",\n      "sentiment": "neutral",\n      "explanation": "Azerbaijan is mentioned in the context of normalization of relations with Armenia, hence the sentiment is neutral."\n    }\n  ]\n}'), (1066289, '{\n  "entities": [\n    {\n      "name": "United States",\n      "sentiment": "neutral",\n      "explanation": "The United States is still ready for direct talks with Iran on the nuclear deal, which indicates a willingness to engage in diplomacy."\n    },\n    {\n      "name": "Iran

In [ ]:
cur = db.execute("select count(*) from article where feedback is not null")
db.commit()

(l := list(cur))

In [ ]:
cur = db.execute(f'select count(*) from article where content like "%United States%"')
db.commit()

(l := list(cur))

In [ ]:
cur = db.execute(f'select count(*) from article where feedback like "%Russia%"')
db.commit()

(l := list(cur))

In [ ]:
cur = db.execute(f'select count(*) from article where headline like "%Russia%"')
db.commit()

(l := list(cur))